# **Character Code**
* 캐릭터 정보를 입력받아 캐릭터 코드를 반환받는 스크립트입니다.
* 쿼리 작성 후, 원하는 캐릭터 정보를 확인한 후 캐릭터 코드를 요청합니다.

### **Import**

In [1]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.api_request import DNF_API
from configs.config import MYSQL_CONNECTION_STRING
from configs.config import DATA_PATH
from configs.config import API_KEYS
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from ratelimit import limits, sleep_and_retry
from threading import BoundedSemaphore
from glob import glob
from tqdm import tqdm
import pandas as pd
import os

# * 오전 06:00 전 데이터는 이전 날짜로 취급합니다.
date = (datetime.now() - timedelta(hours=6)).strftime('%Y%m%d')
engine = create_engine(MYSQL_CONNECTION_STRING)
loaders = [DNF_API(api_key) for api_key in API_KEYS]

%load_ext sql
%sql {MYSQL_CONNECTION_STRING}

### **캐릭터 코드 불러오기 - 멀티스레드**
* 요청받은 직업의 캐릭터 정보들을 이용해 캐릭터 코드를 불러옵니다.
* 속도를 높이기 위해 멀티스레드를 사용합니다.

In [2]:

semaphore = BoundedSemaphore(value=20)
ONE_SECOND = 1

@sleep_and_retry
@limits(calls=110, period=ONE_SECOND)
def request_char_code(loader: DNF_API, sv_eng: str, char_encoded_name: str):
    
    """
    ### Summary
        - 캐릭터의 고유 id를 반환합니다.

    ### Args
        - loader (DNF_API) : DNF_API 인스턴스
        - sv_eng (str) : 캐릭터 서버 (영문)
        - char_name (str) : 캐릭터 이름 (인코딩 시 오류 감소)
        
    Returns:
        characterId (str) : 캐릭터 코드 반환. 에러 발생시 None
    """
    
    try:
        code = loader.character_search(sv_eng, char_encoded_name)
        return code['characterId'][0]
    except Exception as e:
        return None


def character_code(loaders: list, job_name: str, request_list: list, thread_num: int) -> list:

    """
    ### Summary
        - 캐릭터 정보 (영문 서버, 캐릭터 이름)들을 입력받아 캐릭터 코드를 반환하는 함수

    ### Args
        - loaders (list) : DNF 컨테이너가 담긴 리스트
        - request_list (list[str,...,str]) : 요청할 캐릭터 정보
        
    Returns:
        - result (list) : 캐릭터 코드 정보
    """

    # step 1: 요청 리스트에 길이에 맞는 결과 리스트 생성
    L = len(request_list)
    result = [None]*L

    with ThreadPoolExecutor(max_workers = thread_num) as executor:
        
        futures = []
        for idx,(sv_eng,char_encoded_name) in enumerate(request_list):
            
            # step 2: 코드 요청하기
            # * round robbin으로 loader 돌아가면서 사용
            loader = loaders[idx%len(loaders)]
            future = executor.submit(request_char_code,
                                    loader, sv_eng, char_encoded_name)
            futures.append((future, idx))

        # step 3: 값 저장 및 진행상황 출력
        for future, idx in tqdm(futures, total=len(futures), desc=f"{job_name} 처리 중"):
            try:
                code = future.result()
                result[idx] = code
                    
            except Exception as e:
                result[idx] = None
    
    return result

#### **캐릭터 코드 추출**

In [ ]:


# * 크롤링 날짜에 해당하는 캐릭터 정보 불러오기
folder_path = os.path.join(DATA_PATH, 'crawling_data', f'{date}')
csv_files = glob(os.path.join(folder_path, '*.csv'))

for file in csv_files:
    
    # step 1 : 요청할 서버, 캐릭터 이름 정보 불러오기
    job_name = file.split('\\')[-1][:-4]
    
    df = pd.read_csv(file, encoding='utf-8')
    request_list = [tuple(row) for row in df[['sv_eng', 'char_name_encoded']].values]
    
    # step 2 : 캐릭터 코드 불러오기
    df['char_code'] = character_code(loaders, job_name, request_list, 16)
    
    # step 3 : 로컬 저장
    df.to_csv(file, index=False, encoding='utf-8')
